In [1]:
# !pip install geojson
# !pip install folium
# !pip install geopandas
# !pip install pyogrio

In [1]:
import matplotlib.pyplot as plt 
import matplotlib.patches as mpatches
import json
import folium
import geopandas as gpd
import pyogrio
from shapely.ops import split
from shapely.geometry import MultiLineString, LineString, Point, Polygon, MultiPolygon
import pandas as pd
import numpy as np
from folium.plugins import MarkerCluster
import json
import pandas as pd
import branca

In [2]:
from utils_data import get_contours_city_ign, get_iris_contours_city, add_population_iris, get_vehicles_city , add_area_gdf, add_revenus_iris

In [24]:
# estimation moyennes : consommation d'un habitant qui roule en EV 
code_insee = '44055' # Identifiant qui a l'air unique pour IGN
avg_dist_hab = 30
conso_km = 0.2
avg_demand_hab = avg_dist_hab * conso_km
crs_meters = 'EPSG:2154'
crs_angles = 'EPSG:4326'

In [27]:
import os

data_dir = '../data'

file_ign_region = f'{data_dir}/BDCARTO/44_Loire_Atlantique/data.gpkg'
file_iris_contours = f'{data_dir}/IRIS/contours-iris.gpkg'
file_iris_population = f"{data_dir}/IRIS/base-ic-logement-2021.xlsx"
file_iris_revenus = f"{data_dir}/IRIS/iris_revenus.csv"

dir_ev = f'{data_dir}/vehicules'
file_vp = f"{dir_ev}/parc_vp_com_2011_2024.xlsx"
file_vul = f"{dir_ev}/parc_vul_com_2011_2024.xlsx"
file_pl = f"{dir_ev}/parc_pl_com2011_2024.xlsx"
file_tcp = f"{dir_ev}/parc_tcp_commune_2024.xlsx"

# A parametrer selon le code de la ville
# processed_dir = f'{data_dir}/processed'
# os.makedirs(processed_dir, exist_ok=True)

preprocessed_dir = f'{data_dir}/preprocessed'
os.makedirs(preprocessed_dir, exist_ok=True)

processed_dir = f'{data_dir}/processed'
os.makedirs(processed_dir, exist_ok=True)

preprocessed_file_vp = f"{preprocessed_dir}/vp_{code_insee}.xlsx"
preprocessed_file_vul = f"{preprocessed_dir}/vul_{code_insee}.xlsx"
preprocessed_file_pl = f"{preprocessed_dir}/pl_{code_insee}.xlsx"
preprocessed_file_tcp = f"{preprocessed_dir}/tcp_{code_insee}.xlsx"
processed_files_vehicules = {'vp': preprocessed_file_vp, 'vul': preprocessed_file_vul, 'pl': preprocessed_file_pl, 'tcp': preprocessed_file_tcp}

files_vehicules = {'vp': file_vp, 'vul': file_vul, 'pl': file_pl, 'tcp': file_tcp}
export_file_name = f"{processed_dir}/df_vehicles_{code_insee}_2024.csv"

export_file_iris_pop_demand = f"{processed_dir}/demand_per_iris_hab_{code_insee}.geojson"

# Comparaison IRIS

In [48]:
gdf_iris = get_iris_contours_city(file_iris_contours, code_insee)
gdf_iris = add_area_gdf(gdf_iris, crs_meters=crs_meters, crs_angles=crs_angles)
gdf_iris = add_revenus_iris(file_iris_revenus, gdf_iris)

In [50]:
gdf_iris_population = add_population_iris(file_iris_population, gdf_iris)
gdf_iris_population['density_km2'] = gdf_iris_population['population'] / gdf_iris_population['area_km2']
gdf_iris_population.head()

,code_iris,nom_iris,geometry,area_km2,revenues,population,ratio_appart,density_km2
0,440550101,La Baule les Pins,"MULTIPOLYGON (((-2.35133 47.27244, -2.35149 47...",1.818968,31760,2346.994576,0.761234,1290.289305
1,440550102,Centre-Benoît,"MULTIPOLYGON (((-2.38127 47.27993, -2.38559 47...",1.570374,31940,1616.919056,0.766357,1029.639154
2,440550103,Gare-Grand Clos,"MULTIPOLYGON (((-2.4273 47.27833, -2.42724 47....",1.179713,26750,1869.032071,0.655091,1584.310324
3,440550104,Beslon,"MULTIPOLYGON (((-2.40491 47.28723, -2.40464 47...",2.277268,23160,2408.488869,0.330734,1057.621831
4,440550105,Escoublac,"MULTIPOLYGON (((-2.38977 47.30009, -2.38918 47...",8.330503,27110,3511.317978,0.240380,421.501327


In [51]:
pop_city = gdf_iris_population.population.sum() # population de la ville
df_vehicules = get_vehicles_city(code_insee, files_vehicules, processed_files_vehicules, export_file_name)
ratio_ev_pop = ((df_vehicules['NB_EL']+df_vehicules['NB_RECHARGEABLE'])/pop_city).iloc[0]

File already exists


In [52]:
gdf_iris_population['demand_pop_kWh'] = gdf_iris_population['population'] * ratio_ev_pop * gdf_iris_population['ratio_appart'] * avg_demand_hab
# gdf_iris_population.drop(columns=['P21_PMEN', 'ratio_appart'], inplace=True)
gdf_iris_population.drop(columns=['ratio_appart', 'area_km2'], inplace=True)
gdf_iris_population.to_file(export_file_iris_pop_demand, driver="GeoJSON")
gdf_iris_population.head()

,code_iris,nom_iris,geometry,revenues,population,density_km2,demand_pop_kWh
0,440550101,La Baule les Pins,"MULTIPOLYGON (((-2.35133 47.27244, -2.35149 47...",31760,2346.994576,1290.289305,463.191521
1,440550102,Centre-Benoît,"MULTIPOLYGON (((-2.38127 47.27993, -2.38559 47...",31940,1616.919056,1029.639154,321.254665
2,440550103,Gare-Grand Clos,"MULTIPOLYGON (((-2.4273 47.27833, -2.42724 47....",26750,1869.032071,1584.310324,317.430572
3,440550104,Beslon,"MULTIPOLYGON (((-2.40491 47.28723, -2.40464 47...",23160,2408.488869,1057.621831,206.515939
4,440550105,Escoublac,"MULTIPOLYGON (((-2.38977 47.30009, -2.38918 47...",27110,3511.317978,421.501327,218.825894


In [10]:
# Nb de EV dans la ville 
# note : ratio_ev * ratio_voit_poop = nb_ev / pop > juste besoin d'avoir le nb d'ev dans la ville
# nb_voitures_city = 15267 # à récupérer automatiquement
# ratio_voit_pop = nb_voitures_city / pop_city 
# ratio_ev_baule = 0.046 # 4.6% # a récupérer automatiquement 
# gdf_iris_population['demand_pop_kWh'] = gdf_iris_population['P21_PMEN'] * ratio_voit_pop * ratio_ev_baule * gdf_iris_population['ratio_appart'] * avg_demand_hab

# Plot

In [11]:
gdf_ign_city, geometry_ign_city = get_contours_city_ign(file_ign_region, code_insee)
gdf_ign_city.head()

,code_insee,code_insee_du_departement,code_insee_de_la_region,population,surface_en_ha,code_postal,nom_officiel,geometry
266,44055,44,52,16468,2219,44500,La Baule-Escoublac,"MULTIPOLYGON (((-2.29702 47.3032, -2.2972 47.3..."


In [12]:
center_lat, center_lon = geometry_ign_city.centroid.y, geometry_ign_city.centroid.x
m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Create a colormap based on the number of people in each carreau
min_demand = gdf_iris_population['demand_pop_kWh'].min()  # Minimum number of people
max_demand = gdf_iris_population['demand_pop_kWh'].max()  # Maximum number of people
colormap = branca.colormap.linear.YlOrRd_09.scale(min_demand, max_demand)  # Yellow to red color scale

# Add the colormap legend to the map
colormap.add_to(m)

# Iterate through each row in the GeoDataFrame and plot each feature
for _, row in gdf_iris_population.iterrows():
    # Extract geometry and properties for each feature
    feature = row['geometry']
    demand = row['demand_pop_kWh']  # Assuming this column exists
    nom = row['nom_iris']  # Assuming this column exists

    # Create a popup displaying the number of people
    text = f"{nom} : Populatio demand={demand:.2f}kWh"
    popup = folium.Popup(text, max_width=300)
    
    # Style each feature based on the number of people
    folium.GeoJson(
        feature,
        name=f"Carreau {row.name}",
        popup=popup,
        style_function=lambda x, demand=demand: {
                'fillColor': colormap(demand),  # Color based on the number of people
                'color': 'black',  # Border color
                'weight': 2,  # Border width
                'fillOpacity': 0.7  # Fill opacity
            }
    ).add_to(m)
    
# Ajouter un contrôle des couches
folium.LayerControl().add_to(m)

# Afficher la carte
m